# 원본 코드

In [6]:
import zipfile
import os

# 압축 파일의 경로를 지정합니다. 예를 들어 'archive.zip'이라고 가정합니다.
zip_path = '/content/drive/MyDrive/all_wav_test.zip'

# 압축을 해제할 폴더의 경로를 지정합니다.
extract_folder = '/content/data'

# 압축 파일명에서 확장자를 제거하여 압축 해제 폴더 내에 하위 폴더 이름을 생성합니다.
folder_name = os.path.splitext(os.path.basename(zip_path))[0]
extract_path = os.path.join(extract_folder, folder_name)

# 새 폴더를 생성합니다.
os.makedirs(extract_path, exist_ok=True)

# 압축을 해제합니다.
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [8]:
# 압축 파일의 경로를 지정합니다. 예를 들어 'archive.zip'이라고 가정합니다.
zip_path = '/content/drive/MyDrive/voice_output -test.zip'

# 압축을 해제할 폴더의 경로를 지정합니다.
extract_folder = '/content/data'

# 압축 파일명에서 확장자를 제거하여 압축 해제 폴더 내에 하위 폴더 이름을 생성합니다.
folder_name = os.path.splitext(os.path.basename(zip_path))[0]
extract_path = os.path.join(extract_folder, folder_name)

# 새 폴더를 생성합니다.
os.makedirs(extract_path, exist_ok=True)

# 압축을 해제합니다.
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



In [32]:
import librosa
import numpy as np
import os

# 스펙트로그램 패딩 함수
def pad_spectrogram(spectrogram, target_shape):
    """
    주어진 스펙트로그램을 target_shape으로 패딩하거나 자릅니다.
    spectrogram: 원본 스펙트로그램
    target_shape: 목표 형태 (빈도 수, 시간 프레임)
    """
    freq_bins, time_frames = spectrogram.shape
    target_freq_bins, target_time_frames = target_shape
    padded_spectrogram = np.zeros(target_shape)

    # 패딩 또는 자르기 시작 인덱스 계산
    freq_start = (target_freq_bins - freq_bins) // 2
    time_start = (target_time_frames - time_frames) // 2

    if time_frames > target_time_frames:  # 원본이 더 큰 경우, 자르기
        crop_start = (time_frames - target_time_frames) // 2
        spectrogram = spectrogram[:, crop_start:crop_start+target_time_frames]
    elif time_frames < target_time_frames:  # 목표가 더 큰 경우, 패딩
        padded_spectrogram[:, time_start:time_start+time_frames] = spectrogram
        return padded_spectrogram

    if freq_bins > target_freq_bins:  # 원본이 더 큰 경우, 자르기
        crop_start = (freq_bins - target_freq_bins) // 2
        spectrogram = spectrogram[crop_start:crop_start+target_freq_bins, :]
    elif freq_bins < target_freq_bins:  # 목표가 더 큰 경우, 패딩
        padded_spectrogram[freq_start:freq_start+freq_bins, :] = spectrogram
        return padded_spectrogram

    return spectrogram



In [33]:
# 2. 데이터 전처리 함수 정의
def preprocess_audio_folder(folder_path, label, target_shape):
    """
    폴더 내의 모든 오디오 파일을 전처리하고, 스펙트로그램과 라벨을 반환합니다.
    folder_path: 오디오 파일이 있는 폴더 경로
    label: 할당할 라벨
    target_shape: 패딩할 스펙트로그램의 형태
    """
    audio_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    spectrograms = []
    labels = []
    for file in audio_files:
        file_path = os.path.join(folder_path, file)
        audio, sr = librosa.load(file_path, sr=None)
        spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
        log_spectrogram = librosa.power_to_db(spectrogram)
        padded_spectrogram = pad_spectrogram(log_spectrogram, target_shape)  # 패딩
        spectrograms.append(padded_spectrogram)
        labels.append(label)
    return spectrograms, labels

In [34]:
from tensorflow.keras import layers, models

# 2. 모델 구축 함수 정의
def build_model(input_shape):
    """
    Xception 스타일의 모델을 구축합니다.
    input_shape: 입력 데이터의 형태 (빈도 수, 시간 프레임, 채널 수)
    """
    model = models.Sequential()

    # Add the Xception base model
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))

    # Add more layers as per Xception architecture...

    # Add classification head
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification

    return model


In [35]:
# 폴더 경로 및 목표 형태 설정
real_folder_path = '/content/data/all_wav_test'
generated_folder_path = '/content/data/voice_output -test'
target_shape = (128, 300)  # 예시 형태, 조절 필요

# 데이터 전처리
real_spectrograms, real_labels = preprocess_audio_folder(real_folder_path, label=0, target_shape=target_shape)
generated_spectrograms, generated_labels = preprocess_audio_folder(generated_folder_path, label=1, target_shape=target_shape)

# 데이터 결합 및 NumPy 배열로 변환
spectrograms = real_spectrograms + generated_spectrograms
labels = real_labels + generated_labels
X = np.array(spectrograms)[..., np.newaxis]  # 채널 차원 추가
y = np.array(labels)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 구축 및 컴파일
input_shape = (target_shape[0], target_shape[1], 1)
model = build_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
34/34 [==============================] - 12s 32ms/step - loss: 0.4202 - accuracy: 0.8777 - val_loss: 1.1002 - val_accuracy: 0.6556
Epoch 2/20
34/34 [==============================] - 0s 14ms/step - loss: 0.2692 - accuracy: 0.9184 - val_loss: 0.6211 - val_accuracy: 0.7296
Epoch 3/20
34/34 [==============================] - 0s 12ms/step - loss: 0.2445 - accuracy: 0.9157 - val_loss: 0.4890 - val_accuracy: 0.8185
Epoch 4/20
34/34 [==============================] - 0s 11ms/step - loss: 0.2315 - accuracy: 0.9212 - val_loss: 0.3332 - val_accuracy: 0.8630
Epoch 5/20
34/34 [==============================] - 0s 12ms/step - loss: 0.2230 - accuracy: 0.9231 - val_loss: 0.2705 - val_accuracy: 0.8926
Epoch 6/20
34/34 [==============================] - 0s 12ms/step - loss: 0.2100 - accuracy: 0.9240 - val_loss: 0.2597 - val_accuracy: 0.8963
Epoch 7/20
34/34 [==============================] - 0s 11ms/step - loss: 0.2065 - accuracy: 0.9249 - val_loss: 0.2207 - val_accuracy: 0.9148
Epoch 8/20
3

In [36]:
#모델 저장
model.save("my_model.h5")  # HDF5 형식으로 저장

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#추론하는 코드
def predict(audio_file_path, model, target_shape):
    """
    주어진 오디오 파일에 대해 모델을 사용하여 예측하고 결과를 출력합니다.
    audio_file_path: 오디오 파일 경로
    model: 사용할 학습된 모델
    target_shape: 스펙트로그램의 목표 형태
    """
    # 오디오 파일 로드 및 스펙트로그램 생성
    audio, sr = librosa.load(audio_file_path, sr=None)
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    log_spectrogram = librosa.power_to_db(spectrogram)

    # 스펙트로그램 패딩
    padded_spectrogram = pad_spectrogram(log_spectrogram, target_shape)

    # 배치 차원 및 채널 차원 추가
    spectrogram_input = np.expand_dims(padded_spectrogram, axis=[0, -1])

    # 모델을 사용하여 예측
    prediction = model.predict(spectrogram_input)

    # 예측 결과 해석
    if prediction[0] > 0.5:
        print("The audio is predicted to be generated.")
    else:
        print("The audio is predicted to be real.")

# 사용 예시
# predict("path_to_your_audio_file.wav", model, target_shape=(128, 300))


# 모델 추론

# Validation

In [68]:
import librosa
import numpy as np
import math
import json
import os
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, Sequence


class Dataloader(Sequence):
    def __init__(self, Audios, labels, batch_size, target_shape):
        self.Audios = Audios
        self.labels = labels
        self.target_shape = target_shape
        self.batch_size = batch_size
        self.num_classes = len(set(self.labels))
        self.indices = np.arange(len(self.labels))

    def __len__(self):
        return math.ceil(len(self.labels)/self.batch_size)
    
    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_x = [self.Audios[i] for i in indices]
        batch_audios = self.get_Audios(batch_x)
        batch_y = [self.labels[i] for i in indices]
        # 라벨을 원-핫 인코딩
        batch_y = to_categorical(batch_y, num_classes=self.num_classes)
        return np.array(batch_audios), np.array(batch_y)
    
    # 스펙트로그램 패딩 함수
    def pad_spectrogram(self, spectrogram):
        """
        주어진 스펙트로그램을 target_shape으로 패딩하거나 자릅니다.
        spectrogram: 원본 스펙트로그램
        target_shape: 목표 형태 (빈도 수, 시간 프레임)
        """
        freq_bins, time_frames = spectrogram.shape
        target_freq_bins, target_time_frames = self.target_shape
        padded_spectrogram = np.zeros(self.target_shape)

        # 패딩 또는 자르기 시작 인덱스 계산
        freq_start = (target_freq_bins - freq_bins) // 2
        time_start = (target_time_frames - time_frames) // 2

        if time_frames > target_time_frames:  # 원본이 더 큰 경우, 자르기
            crop_start = (time_frames - target_time_frames) // 2
            spectrogram = spectrogram[:, crop_start:crop_start+target_time_frames]
        elif time_frames < target_time_frames:  # 목표가 더 큰 경우, 패딩
            padded_spectrogram[:, time_start:time_start+time_frames] = spectrogram
            return padded_spectrogram

        if freq_bins > target_freq_bins:  # 원본이 더 큰 경우, 자르기
            crop_start = (freq_bins - target_freq_bins) // 2
            spectrogram = spectrogram[crop_start:crop_start+target_freq_bins, :]
        elif freq_bins < target_freq_bins:  # 목표가 더 큰 경우, 패딩
            padded_spectrogram[freq_start:freq_start+freq_bins, :] = spectrogram
            return padded_spectrogram

        return spectrogram


    def get_Audios(self, path_list):
        # 오디오 데이터 로딩 및 전처리
        spectrograms = []
        for file_path in path_list:
            audio, sr = librosa.load(file_path, sr=None)
            spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
            log_spectrogram = librosa.power_to_db(spectrogram)
            padded_spectrogram = self.pad_spectrogram(log_spectrogram)  # 패딩
            spectrograms.append(padded_spectrogram)
            #spectrograms.append(log_spectrogram)
            labels.append(label)

        return np.array(spectrograms)[..., np.newaxis]  # 채널 차원 추가


epochs = 30
target_shape = (128, 300) # 예시 형태, 조절 필요

# JSON 파일 로딩 및 데이터 및 라벨 생성
with open('../test_dataset.json', 'r') as f:
    folder_label_mapping = json.load(f)

file_paths = []
labels = []

# 각 폴더 및 라벨에 대해
for file_path, label in folder_label_mapping.items():
    # 파일 확장자 확인하여 wav 파일만 처리
    if file_path.lower().endswith('.wav'):
        # 파일 경로 및 라벨 저장
        file_paths.append(file_path)
        labels.append(label)

Valid_dataloader = Dataloader(file_paths, labels, 16, target_shape)

In [69]:
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import load_model

model_path = '/root/Compare_models/xception_50epochs_model3.h5'
model = load_model(model_path)
print(tf.__version__)

pred_list = []
label_list = []

for idx in range(Valid_dataloader.__len__()):
    try: 
        data, label = Valid_dataloader.__getitem__(idx)
        pred = model.predict(data)

    except:
        print(idx)
        continue
        
    pred_list.append(to_categorical(np.argmax(pred, axis=1)))
    label_list.append(label)

2.14.0
1/1 [==============================] - 0s 55ms/step


In [70]:
def list_sum(x):
    temp = []
    for i in x:
        if len(temp) < 1:
            temp = i
        else:
            temp = np.concatenate((temp, i))
    return np.array(temp)

In [71]:
concat_label_list = list_sum(label_list)
concat_pred_list = list_sum(pred_list)
print(concat_label_list.shape)
print(concat_pred_list.shape)

(4141, 2)
(4141, 2)


In [72]:
metric = tf.keras.metrics.F1Score(threshold=0.5)
#y_true = to_categorical(labels)
#y_pred = preds
metric.update_state(concat_label_list, concat_pred_list)
result = metric.result()
result.numpy()

array([0.99827963, 0.9983385 ], dtype=float32)

In [73]:
 from sklearn.metrics import f1_score

f1 = f1_score(concat_label_list, concat_pred_list, average='macro')
print("f1_score: %.2f%%" % (f1 * 100.0))



f1_score: 99.83%


In [66]:
# 분류 보고서 출력
report = classification_report(concat_label_list, concat_pred_list)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2031
           1       1.00      1.00      1.00      2110

   micro avg       1.00      1.00      1.00      4141
   macro avg       1.00      1.00      1.00      4141
weighted avg       1.00      1.00      1.00      4141
 samples avg       1.00      1.00      1.00      4141



In [67]:
preds_list_max = np.argmax(concat_pred_list, axis=1)
labels_list_max = np.argmax(concat_label_list, axis=1)

# 정확도 계산
accuracy = np.mean(preds_list_max == labels_list_max)
print(f'Accuracy: {accuracy}')

# 손실 계산 (예를 들면, 교차 엔트로피 손실)
loss = -np.mean(labels_list_max * np.log(preds_list_max) + (1 - labels_list_max) * np.log(1 - preds_list_max))
print(f'Loss: {loss}')

Accuracy: 0.9983095870562666
Loss: nan


/tmp/ipykernel_27827/3133315849.py:9: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(labels_list_max * np.log(preds_list_max) + (1 - labels_list_max) * np.log(1 - preds_list_max))
/tmp/ipykernel_27827/3133315849.py:9: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(labels_list_max * np.log(preds_list_max) + (1 - labels_list_max) * np.log(1 - preds_list_max))
